for Swati-san's microglia data, and practice for MicrogliaTrackingGUI development

In [11]:
import sys
import os

dir_notebook = os.path.dirname(os.path.abspath("__file__"))
# 親ディレクトリのパスを取得
dir_parent = os.path.dirname(dir_notebook)
if not dir_parent in sys.path:
    sys.path.append(dir_parent)

from PyQt5.QtWidgets import QMainWindow, QWidget, QGridLayout, QVBoxLayout, QHBoxLayout, QApplication
from optic.config import *
from optic.controls import *
from optic.gui import *
from optic.io import *
from optic.manager import *
from optic.gui.bind_func import *

class Suite2pROICheckGUI(QMainWindow):
    def __init__(self):
        APP_NAME = "SUITE2P_ROI_CHECK"
        QMainWindow.__init__(self)
        self.widget_manager, self.config_manager, self.data_manager, self.control_manager, self.layout_manager = initManagers(
            WidgetManager(), ConfigManager(), DataManager(), ControlManager(), LayoutManager()
        )
        self.config_manager.setCurrentApp(APP_NAME)
        self.app_keys = self.config_manager.gui_defaults["APP_KEYS"]
        self.app_key_pri = self.app_keys[0]

        self.setupUI_done = False
        setupMainWindow(self, self.config_manager.gui_defaults)

        self.initUI()

    """
    setup UI Function
    """
    def initUI(self):
        self.central_widget = QWidget(self)
        self.setCentralWidget(self.central_widget)
        self.layout_main = QGridLayout(self.central_widget)

        # FileLoadUI用のレイアウト
        self.layout_file_load = QVBoxLayout()
        self.setupFileLoadUI()
        self.layout_main.addLayout(self.layout_file_load, 1, 0, 1, 1)

        # メインUI用のレイアウト
        self.layout_main_ui = QGridLayout()
        self.layout_main.addLayout(self.layout_main_ui, 0, 0, 1, 1)

    def setupFileLoadUI(self):
        file_load_widget = QWidget()
        layout = QVBoxLayout(file_load_widget)
        # ファイル読み込み用のUIを追加
        layout.addLayout(self.makeLayoutSectionBottom())
        # bindFunc
        self.bindFuncFileLoadUI()

        self.layout_file_load.addWidget(file_load_widget)

    def loadFilePathsandInitialize(self):
        self.control_manager, self.data_manager = initManagers(self.control_manager, self.data_manager)
        success = self.loadData()
        if success:
            self.setupMainUI()
        else:
            return

    def setupMainUI(self):
        if self.setupUI_done:
            # メインUIのクリア
            clearLayout(self.layout_main_ui)
        
        # 新しいメインUIの設定
        self.setupMainUILayouts()
        self.setupControls()
        self.bindFuncAllWidget()

        self.setupUI_done = True

    def loadData(self):
        return

    def setupMainUILayouts(self):
        pass

    def setupControls(self):
        # self.control_manager.table_controls[self.app_key_pri].setupWidgetROITable(self.app_key_pri)
        # self.control_manager.view_controls[self.app_key_pri] = ViewControl(
        #     key_app=self.app_key_pri,
        #     q_view=self.widget_manager.dict_view[self.app_key_pri], 
        #     q_scene=self.widget_manager.dict_scene[self.app_key_pri], 
        #     data_manager=self.data_manager, 
        #     widget_manager=self.widget_manager,
        #     config_manager=self.config_manager,
        #     control_manager=self.control_manager,
        # )
        # self.control_manager.view_controls[self.app_key_pri].setViewSize()
        pass

    """
    makeLayout Function; Component
    小要素のLayout
    return -> Layout
    """
    "Bottom"
    def makeLayoutComponentFileLoadUI(self):
        layout = QVBoxLayout()

        # LineEdit
        list_label = ["Fall mat file path", "Reference Tiff image file path (optional)"]
        list_key = [f"{self.app_key_pri}_path_fall", f"{self.app_key_pri}_path_reftif"]
        for label, key in zip(list_label, list_key):
            layout.addLayout(makeLayoutLoadFileWidget(
                self.widget_manager, 
                label=label, 
                key_label=key, 
                key_lineedit=key, 
                key_button=key
            ))
        # Button
        layout.addLayout(makeLayoutLoadFileExitHelp(self.widget_manager))
        return layout

    """
    makeLayout Function; Section
    領域レベルの大Layout
    """
    def makeLayoutSectionBottom(self):
        layout = self.makeLayoutComponentFileLoadUI()
        return layout

    """
    bindFunc Function
    配置したwidgetに関数を紐づけ
    """
    def bindFuncFileLoadUI(self):        
        list_key = [f"{self.app_key_pri}_path_fall", f"{self.app_key_pri}_path_reftif"]
        list_filetype = ["mat", "tiff"]
        for key, filetype in zip(list_key, list_filetype):
            bindFuncLoadFileWidget(
                q_widget=self, 
                q_button=self.widget_manager.dict_button[key], 
                q_lineedit=self.widget_manager.dict_lineedit[key], 
                filetype=filetype
            )

        self.widget_manager.dict_button["load_file"].clicked.connect(lambda: self.loadFilePathsandInitialize())
        bindFuncExit(q_window=self, q_button=self.widget_manager.dict_button["exit"])

    def bindFuncAllWidget(self):
        pass


if __name__ == "__main__":
    app = QApplication(sys.argv) if QApplication.instance() is None else QApplication.instance()
    applyAppStyle(app)
    gui = Suite2pROICheckGUI()
    gui.show()
    sys.exit(app.exec_())

SystemExit: 0

c:\Users\HyperPC_Smee\anaconda3\envs\itkelastix\lib\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [7]:
import tifffile
path_tiff = '../data/swati-san/Original_resample.tif'


# TIFFファイルを開く
with tifffile.TiffFile(path_tiff) as tif:
    # 最初のシリーズ（通常は1つだけ）の次元情報を取得
    series = tif.series[0]
    
    # 軸の順序を取得
    axes = series.axes
    
    # 各次元のサイズを取得
    shape = series.shape
    
    # 次元情報を表示
    print(f"Axes order: {axes}")
    print(f"Shape: {shape}")

    # 画像データを読み込む（オプション）
    data = tif.asarray()
    print(f"Data shape: {data.shape}")

Axes order: TZCYX
Shape: (5, 67, 2, 568, 562)
Data shape: (5, 67, 2, 568, 562)


'TZCYX'